<a href="https://colab.research.google.com/github/Rohan-Witty/Implementation/blob/main/ResNet_impl_imagenet_model_for_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
training_data = datasets.CIFAR10(
    root = './data',
    train = True,
    download = True,
    transform = ToTensor(),
)
testing_data = datasets.CIFAR10(
    root = './data',
    train = False,
    download = True,
    transform = ToTensor(),
)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
batch_size = 128

train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(testing_data, batch_size = batch_size)
for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape, X.dtype)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([128, 3, 32, 32]) torch.float32
Shape of y:  torch.Size([128]) torch.int64


In [ ]:
torch.cuda.is_available()

True

In [ ]:
class VggLike(nn.Module):
    def __init__(self):
        super(VggLike, self).__init__()
        self.conv_stack_1 = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size= (7,7), stride = 2, padding=3)
        )
        self.conv_stack_2 = nn.Sequential(
            nn.MaxPool2d(kernel_size= (3,3), stride=2, padding = 1),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
        )
        self.conv_stack_3 = nn.Sequential(
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size= (3,3), stride = 2,padding = 1),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
        )
        self.conv_stack_4 = nn.Sequential(
            nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size= (3,3), stride = 2, padding = 1),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
        )
        self.conv_stack_5 = nn.Sequential(
            nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size= (3,3), stride = 2, padding = 1),
            nn.BatchNorm2d(512, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(512, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(512, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(512, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size= (3,3), stride =1, ),
        )
        self.fcsoftmax = nn.Sequential(
            nn.Linear(12800,10),
            #nn.Softmax(1),
        )
        self.flatten = nn.Flatten()
        
    def forward(self, x):
        x = self.conv_stack_1(x)
        x = self.conv_stack_2(x)
        x = self.conv_stack_3(x)
        x = self.conv_stack_4(x)
        x = self.conv_stack_5(x)
        x = self.flatten(x)
        logits = self.fcsoftmax(x)
        return logits

In [ ]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.conv_stack_1 = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size= (7,7), stride = 2, padding=3)
        )
        self.conv_stack_2_pool = nn.Sequential(
            nn.MaxPool2d(kernel_size= (3,3), stride=2, padding = 1),
        )
        self.conv_stack_2_1 = nn.Sequential(
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, ),
            nn.ReLU(),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1,),
        )
        self.conv_stack_2_2 = nn.Sequential(    
            nn.ReLU(),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1,),
        )
        self.conv_stack_3_1 = nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size= (3,3), stride = 2,padding = 1),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1,),
        )
        self.conv_projection_1 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=1, stride = 2),
            nn.BatchNorm2d(128),
        )
        self.conv_stack_3_2 = nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1,),
        )
        self.conv_stack_4_1 = nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size= (3,3), stride = 2, padding = 1),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1,),
        )
        self.conv_projection_2 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=1, stride = 2),
            nn.BatchNorm2d(256),
        )
        self.conv_stack_4_2 = nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1,),
        )
        self.conv_stack_5_1 = nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size= (3,3), stride = 2, padding = 1),
            nn.BatchNorm2d(512, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(512, eps=1e-05, momentum=0.1,),
        )
        self.conv_projection_3 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=1, stride = 2),
            nn.BatchNorm2d(512),
        )
        self.conv_stack_5_2 = nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(512, eps=1e-05, momentum=0.1,),
            nn.ReLU(),
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size= (3,3), stride = 1, padding = 'same',),
            nn.BatchNorm2d(512, eps=1e-05, momentum=0.1,),
        )
        self.conv_stack_5_pool = nn.Sequential(            
            nn.ReLU(),
            nn.AvgPool2d(kernel_size= (3,3), stride =1, ),
        )
        self.fcsoftmax = nn.Sequential(
            nn.Linear(12800,10),
            #nn.Softmax(1),
        )
        self.flatten = nn.Flatten()
        
    def forward(self, x):
        x_1 = self.conv_stack_1(x)
        x_2_pool = self.conv_stack_2_pool(x_1)
        x_2_1 = self.conv_stack_2_1(x_2_pool)
        x_2_2 = self.conv_stack_2_2(x_2_1 + x_2_pool)
        x_3_1 = self.conv_stack_3_1(x_2_2 + x_2_1)
  
        x_3_2 = self.conv_stack_3_2(x_3_1 + self.conv_projection_1(x_2_2))
        x_4_1 = self.conv_stack_4_1(x_3_2 + x_3_1)

        x_4_2 = self.conv_stack_4_2(x_4_1 + self.conv_projection_2(x_3_2))
        x_5_1 = self.conv_stack_5_1(x_4_2 + x_4_1)
        
        x_5_2 = self.conv_stack_5_2(x_5_1 + self.conv_projection_3(x_4_2))
        out = self.flatten(self.conv_stack_5_pool(x_5_2))
        logits = self.fcsoftmax(out)
        return logits

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = VggLike().to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/Implementations/Cifar1.pth',map_location=torch.device(device)))
print(model)

model_res = ResNet().to(device)
model_res.load_state_dict(torch.load('/content/drive/MyDrive/Implementations/Cifar10resnet.pth',map_location=torch.device(device)))
print(model_res)

In [ ]:
from torchsummary import summary
summary(model, input_size=(3, 224, 224))

summary(model_res, input_size = (3,224,224))

In [ ]:
loss_fn = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model_res.parameters(), lr=0.1,momentum = .9, weight_decay=.0001 ) Used for first 25 epochs in resnet and first 45 epochs in plain vgg-like
optimizer = torch.optim.AdamW(model_res.parameters(), lr = 3e-3, weight_decay = 0.1)

In [ ]:
from torchvision.transforms import Resize, RandomHorizontalFlip, RandomCrop
def preprocess(X):
    X = RandomHorizontalFlip(p=0.5)(RandomCrop(224)(Resize(256)(X)))
    return X

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    print("training size is :", size)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        X = preprocess(X)
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    print("training size is :", size)
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            X = preprocess(X)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
def train_error(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    print("training size is :", size)
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            X = preprocess(X)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Train Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 20

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model_res, loss_fn, optimizer)
    test(test_dataloader, model_res, loss_fn)
print("Done!")

Epoch 1
-------------------------------
training size is : 50000
loss: 0.183913  [    0/50000]
loss: 0.444872  [12800/50000]
loss: 0.708959  [25600/50000]
loss: 0.433231  [38400/50000]
training size is : 10000
Test Error: 
 Accuracy: 78.4%, Avg loss: 0.624970 

Epoch 2
-------------------------------
training size is : 50000
loss: 0.314498  [    0/50000]
loss: 0.393963  [12800/50000]
loss: 0.637904  [25600/50000]
loss: 0.568792  [38400/50000]
training size is : 10000
Test Error: 
 Accuracy: 80.1%, Avg loss: 0.593656 

Epoch 3
-------------------------------
training size is : 50000
loss: 0.291631  [    0/50000]
loss: 0.403327  [12800/50000]
loss: 0.495426  [25600/50000]
loss: 0.417319  [38400/50000]
training size is : 10000
Test Error: 
 Accuracy: 80.5%, Avg loss: 0.577503 

Epoch 4
-------------------------------
training size is : 50000
loss: 0.263431  [    0/50000]
loss: 0.254238  [12800/50000]
loss: 0.423727  [25600/50000]
loss: 0.419049  [38400/50000]
training size is : 10000
Test

In [ ]:
torch.save(model_res.state_dict(), "/content/drive/MyDrive/Implementations/Cifar10resnet.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [ ]:
print("For RESNET MODEL:\n")
train_error(train_dataloader, model_res, loss_fn)
test(test_dataloader, model_res, loss_fn)

For RESNET MODEL:

training size is : 50000
Train Error: 
 Accuracy: 89.4%, Avg loss: 0.309982 

training size is : 10000
Test Error: 
 Accuracy: 84.9%, Avg loss: 0.482919 



In [ ]:
print("For VGG-Like PLAIN MODEL:\n")
train_error(train_dataloader, model, loss_fn)
test(test_dataloader, model, loss_fn)

For VGG-Like PLAIN MODEL:

training size is : 50000
Train Error: 
 Accuracy: 88.3%, Avg loss: 0.347580 

training size is : 10000
Test Error: 
 Accuracy: 80.1%, Avg loss: 0.721762 



Training of epochs 6-25 for Resnet
Epoch 1
-------------------------------
training size is : 50000
loss: 1.480535  [    0/50000]
loss: 1.448804  [12800/50000]
loss: 1.507441  [25600/50000]
loss: 1.364420  [38400/50000]
training size is : 50000
Test Error: 
 Accuracy: 49.2%, Avg loss: 1.405268 

Epoch 2
-------------------------------
training size is : 50000
loss: 1.390570  [    0/50000]
loss: 1.357148  [12800/50000]
loss: 1.328467  [25600/50000]
loss: 1.343819  [38400/50000]
training size is : 50000
Test Error: 
 Accuracy: 45.5%, Avg loss: 1.517867 

Epoch 3
-------------------------------
training size is : 50000
loss: 1.213826  [    0/50000]
loss: 1.197033  [12800/50000]
loss: 1.302036  [25600/50000]
loss: 1.215209  [38400/50000]
training size is : 50000
Test Error: 
 Accuracy: 52.5%, Avg loss: 1.322819 

Epoch 4
-------------------------------
training size is : 50000
loss: 1.290869  [    0/50000]
loss: 1.021148  [12800/50000]
loss: 1.184215  [25600/50000]
loss: 1.079633  [38400/50000]
training size is : 50000
Test Error: 
 Accuracy: 54.9%, Avg loss: 1.289782 

Epoch 5
-------------------------------
training size is : 50000
loss: 1.121272  [    0/50000]
loss: 1.062406  [12800/50000]
loss: 1.120097  [25600/50000]
loss: 0.952055  [38400/50000]
training size is : 50000
Test Error: 
 Accuracy: 54.0%, Avg loss: 1.342604 

Epoch 6
-------------------------------
training size is : 50000
loss: 1.063403  [    0/50000]
loss: 0.953306  [12800/50000]
loss: 0.988814  [25600/50000]
loss: 0.880686  [38400/50000]
training size is : 50000
Test Error: 
 Accuracy: 58.7%, Avg loss: 1.196016 

Epoch 7
-------------------------------
training size is : 50000
loss: 0.838953  [    0/50000]
loss: 0.832382  [12800/50000]
loss: 0.881330  [25600/50000]
loss: 0.795400  [38400/50000]
training size is : 50000
Test Error: 
 Accuracy: 53.6%, Avg loss: 1.415353 

Epoch 8
-------------------------------
training size is : 50000
loss: 0.840478  [    0/50000]
loss: 0.825738  [12800/50000]
loss: 0.867948  [25600/50000]
loss: 0.732122  [38400/50000]
training size is : 50000
Test Error: 
 Accuracy: 67.0%, Avg loss: 0.966242 

Epoch 9
-------------------------------
training size is : 50000
loss: 0.721857  [    0/50000]
loss: 0.792052  [12800/50000]
loss: 0.680949  [25600/50000]
loss: 0.626377  [38400/50000]
training size is : 50000
Test Error: 
 Accuracy: 72.7%, Avg loss: 0.784339 

Epoch 10
-------------------------------
training size is : 50000
loss: 0.600122  [    0/50000]
loss: 0.632830  [12800/50000]
loss: 0.668864  [25600/50000]
loss: 0.645847  [38400/50000]
training size is : 50000
Test Error: 
 Accuracy: 73.5%, Avg loss: 0.768271 

Epoch 11
-------------------------------
training size is : 50000
loss: 0.478513  [    0/50000]
loss: 0.576744  [12800/50000]
loss: 0.590296  [25600/50000]
loss: 0.507258  [38400/50000]
training size is : 50000
Test Error: 
 Accuracy: 78.1%, Avg loss: 0.634943 

Epoch 12
-------------------------------
training size is : 50000
loss: 0.520497  [    0/50000]
loss: 0.531903  [12800/50000]
loss: 0.547542  [25600/50000]
loss: 0.498417  [38400/50000]
training size is : 50000
Test Error: 
 Accuracy: 75.9%, Avg loss: 0.704336 

Epoch 13
-------------------------------
training size is : 50000
loss: 0.434589  [    0/50000]
loss: 0.359435  [12800/50000]
loss: 0.578484  [25600/50000]
loss: 0.534104  [38400/50000]
training size is : 50000
Test Error: 
 Accuracy: 81.8%, Avg loss: 0.528547 

Epoch 14
-------------------------------
training size is : 50000
loss: 0.428045  [    0/50000]
loss: 0.483676  [12800/50000]
loss: 0.439411  [25600/50000]
loss: 0.368756  [38400/50000]
training size is : 50000
Test Error: 
 Accuracy: 82.3%, Avg loss: 0.521460 

Epoch 15
-------------------------------
training size is : 50000
loss: 0.476259  [    0/50000]
loss: 0.432962  [12800/50000]
loss: 0.408788  [25600/50000]
loss: 0.408962  [38400/50000]
training size is : 50000
Test Error: 
 Accuracy: 82.9%, Avg loss: 0.495774 

Epoch 16
-------------------------------
training size is : 50000
loss: 0.302260  [    0/50000]
loss: 0.352506  [12800/50000]
loss: 0.340625  [25600/50000]
loss: 0.515029  [38400/50000]
training size is : 50000
Test Error: 
 Accuracy: 84.0%, Avg loss: 0.466931 

Epoch 17
-------------------------------
training size is : 50000
loss: 0.270795  [    0/50000]
loss: 0.332236  [12800/50000]
loss: 0.370278  [25600/50000]
loss: 0.368341  [38400/50000]
training size is : 50000
Test Error: 
 Accuracy: 83.6%, Avg loss: 0.482988 

Epoch 18
-------------------------------
training size is : 50000
loss: 0.230397  [    0/50000]
loss: 0.322002  [12800/50000]
loss: 0.650147  [25600/50000]
loss: 0.374658  [38400/50000]
training size is : 50000
Test Error: 
 Accuracy: 84.7%, Avg loss: 0.452296 

Epoch 19
-------------------------------
training size is : 50000
loss: 0.204696  [    0/50000]
loss: 0.308192  [12800/50000]
loss: 0.443483  [25600/50000]
loss: 0.410353  [38400/50000]
training size is : 50000
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.507657 

Epoch 20
-------------------------------
training size is : 50000
loss: 0.279357  [    0/50000]
loss: 0.379822  [12800/50000]
loss: 0.306879  [25600/50000]
loss: 0.275853  [38400/50000]
training size is : 50000
Test Error: 
 Accuracy: 85.9%, Avg loss: 0.418605 

Done!